# PCNA experiments raw data viewer

This notebook takes the output of the 4-channel brightfield, gfp-h2b, rfp-h2b, irfp-pcna experiments, formats the images into stacks and displays the stacks in Napari.

In [1]:
import napari
print("Napari version no.:", napari.__version__)
import btrack
print('Btrack version', btrack.__version__)
import os, glob, enum
import numpy as np
import tifffile as tiff
from tqdm import tqdm
from skimage.io import imread
from natsort import natsorted
from datetime import datetime

Napari version no.: 0.4.7
Btrack version 0.4.0


### Set data folder and number of channels to display

In [25]:
### set root data folder
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
print('Experiment/position list:', [(expt,natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])) for expt in expt_IDs])

### not entirely sure this is the correct use of a class but im still learning
@enum.unique
class channels(enum.Enum):
#     bf = 0 
#     gfp = 1
#     rfp = 2
    irfp = 3

Experiment/position list: [('MK0000', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']), ('MK0001', ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15'])]


In [26]:
expt = 'MK0000'
pos = 'Pos15'

data_folder = os.path.join(root_folder, '{}/{}'.format(expt, pos)) #'/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15'#'/media/nathan/Transcend/Nathan/data/pcna/test_210422/Pos7'

### Compile folders of single frame images into stacks if necessary

In [5]:
## compile stacks
output_dir = os.path.join(data_folder,'{}_stacks'.format(pos))
if '{}_stacks'.format(pos) not in os.listdir(data_folder):
    os.mkdir(os.path.join(data_folder,'{}_stacks'.format(pos)))
    print(output_dir, 'directory created')
    for channel in channels:
        print(channel.name, 'compiling')
        files = glob.glob(os.path.join(data_folder, '{}_raw'.format(pos),'img_channel00{}*.tif'.format(channel.value)))
        stack = []
        for file in tqdm(natsorted(files)):
            tif = np.array(tiff.imread(file),dtype=np.uint8)
            tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
            tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
            stack.append(tif)
        stack = np.stack(stack,axis=0)
        with tiff.TiffWriter(os.path.join(output_dir,'{}.tif'.format(channel.name)),imagej=True) as output_file:
            output_file.save(stack)
        print(channel.name, 'saved out')
else:
    print('looks like stacks already exist bro')

looks like stacks already exist bro


### Load images 

In [6]:
### load images
output_dir = os.path.join(data_folder,'{}_stacks'.format(pos))
stack_fn = {}
for channel in channels:
    stack_fn[channel.name] = glob.glob(os.path.join(output_dir, 'channel00{}.tif'.format(channel.value)))
    #stack_fn[channel.name] = glob.glob(os.path.join(output_dir, '{}.tif'.format(channel.name)))
# gfp = imread(stack_fn['gfp'][0])
# rfp = imread(stack_fn['rfp'][0])
irfp = imread(stack_fn['irfp'][0])

### Load tracks

In [27]:
tracks_path = natsorted(glob.glob('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/{}/{}/*.h5'.format(expt, pos)))
[(i, name) for i,name in enumerate(tracks_path)]

[(0,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/_irfp_tracks.h5'),
 (1,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/gfp_tracks_2021_05_21_05_57_38_PM.h5'),
 (2,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/gfp_tracks_2021_05_28_07_20_04_PM.h5'),
 (3,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5'),
 (4,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_21_05_57_45_PM.h5'),
 (5,
  '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_28_07_20_11_PM.h5')]

In [28]:
gfp_tracks = tracks_path[2]
rfp_tracks = tracks_path[5]
irfp_tracks = tracks_path[3]

with btrack.dataio.HDF5FileHandler(irfp_tracks, 'r', obj_type = "obj_type_2") as hdf:
    wt2_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(gfp_tracks, 'r', obj_type = "obj_type_3") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(rfp_tracks, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

[INFO][2021/05/30 06:53:39 pm] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5...


2021-05-30 18:53:39,114 [INFO] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/irfp_tracks_2021_05_28_07_16_36_PM.h5...


[INFO][2021/05/30 06:53:39 pm] Loading tracks/obj_type_2


2021-05-30 18:53:39,163 [INFO] Loading tracks/obj_type_2


[INFO][2021/05/30 06:53:39 pm] Loading objects/obj_type_2 (551730, 5) (551730 filtered: None)


...

2021-05-30 18:53:52,568 [INFO] Loading tracks/obj_type_2


[INFO][2021/05/30 06:53:52 pm] Loading objects/obj_type_2 (10306, 5) (10306 filtered: None)


2021-05-30 18:53:52,646 [INFO] Loading objects/obj_type_2 (10306, 5) (10306 filtered: None)


[INFO][2021/05/30 06:53:52 pm] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_28_07_20_11_PM.h5


2021-05-30 18:53:52,749 [INFO] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/rfp_tracks_2021_05_28_07_20_11_PM.h5
Tracks loaded


In [29]:
irfp_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy
0,70,0,290.582672,103.409477,0.0,70,70,5,0,False
1,70,1,290.582672,103.409477,0.0,70,70,5,0,True
2,70,2,285.010681,107.029442,0.0,70,70,5,0,False
3,70,3,287.318939,104.421059,0.0,70,70,5,0,False
4,70,4,286.586121,105.112755,0.0,70,70,5,0,False


In [31]:
wt_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy
0,70,0,291.845612,103.778946,0.0,70,70,5,0,False
1,70,1,291.845612,103.778946,0.0,70,70,5,0,True
2,70,2,284.170502,107.681984,0.0,70,70,5,0,False
3,70,3,287.813873,104.059937,0.0,70,70,5,0,False
4,70,4,286.591492,104.917526,0.0,70,70,5,0,False


In [33]:
scr_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy
0,2,0,1344.406738,440.096802,0.0,2,2,5,0,False


In [23]:
cell = [cell for cell in irfp_tracks if cell.ID == 46][0]
cell

,ID,t,x,y,z,parent,root,state,generation,dummy
0,46,0,41.309788,719.011902,0.000000,46,46,5,0,False
1,46,1,43.383797,723.153076,0.000000,46,46,5,0,False
2,46,2,43.559811,723.514465,0.000000,46,46,5,0,False
3,46,3,39.259796,725.185608,0.000000,46,46,5,0,False
4,46,4,35.318588,727.776428,0.000000,46,46,5,0,False
5,46,5,34.297951,729.871094,0.000000,46,46,5,0,False
6,46,6,35.123959,730.044556,0.000000,46,46,5,0,False
7,46,7,38.505508,730.403748,0.000000,46,46,5,0,False
8,46,8,39.235733,730.959412,0.000000,46,46,5,0,False
9,46,9,40.164059,731.671936,0.012751,46,46,5,0,True


In [15]:
y, x = cell.x[0], cell.y[0]

### Launch napari

In [16]:
### launch napari
with napari.gui_qt():
    viewer = napari.Viewer()
    
    ### add images
    #viewer.add_image(bf, name = "brightfield")
    #viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    #viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
    viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")
    
    ## add tracks
    viewer.add_tracks(irfp_tracks)
    #viewer.add_tracks(scr_tracks)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

